# Run MethaneMIP scenarios

Use fair calibration v1.4.0, which is appropriate for SSP scenarios with corrected NOx emissions.

In [ ]:
import logging

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
logger = logging.getLogger('fair')
logger.setLevel(level=logging.CRITICAL)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")

MethaneMIP scenarios are defined to 2050. CMIP7 might run to 2125, so I have extended to 2030 since emissions are given at 10 years

In [ ]:
f.define_time(1750, 2131, 1)

In [ ]:
scenarios = [
    'ssp245',
    'ssp245-ModAggr',
    'ssp245-Aggr',
]

In [ ]:
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_4_0_file = '../data/parameters/calibrated_constrained_parameters_1.4.0.csv'

In [ ]:
df_configs = pd.read_csv(fair_params_1_4_0_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
fair_species_configs_1_4_0_file = '../data/parameters/species_configs_properties_1.4.0.csv'

In [ ]:
species, properties = read_properties(filename=fair_species_configs_1_4_0_file)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_csv(
    emissions_file='../data/emissions/methane-emissions-scenarios.csv',
    forcing_file='../data/forcing/volcanic_solar.csv',
)

In [ ]:
fill(
    f.forcing,
    f.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)

In [ ]:
f.fill_species_configs(fair_species_configs_1_4_0_file)
f.override_defaults(fair_params_1_4_0_file)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.concentration.sel(specie='CH4', scenario='ssp245'), color='r');
pl.plot(f.concentration.sel(specie='CH4', scenario='ssp245-ModAggr'), color='k');
pl.plot(f.concentration.sel(specie='CH4', scenario='ssp245-Aggr'), color='b');
pl.xlim(250, 300)

In [ ]:
pd.DataFrame(
    f.concentration.sel(specie='CH4').interp(timebounds=f.timepoints).median(dim='config'),
    index=f.timepoints,
    columns=f.scenarios
).to_csv('../output/ch4-concentration-medians.csv')

In [ ]:
pl.plot(10 * f.alpha_lifetime.sel(specie='CH4', scenario='ssp245'), color='r');
pl.plot(10 * f.alpha_lifetime.sel(specie='CH4', scenario='ssp245-ModAggr'), color='k');
pl.plot(10 * f.alpha_lifetime.sel(specie='CH4', scenario='ssp245-Aggr'), color='b');
pl.xlim(250, 300)

In [ ]:
weights = np.zeros((382, 3, 841))
weights[100, :, :] = 0.5
weights[101:151, :, :] = 1
weights[151, :, :] = 0.5
weights = xr.DataArray(
    weights, 
    dims=f.temperature.sel(layer=0).dims, 
    coords=f.temperature.sel(layer=0).coords
)

In [ ]:
temp_rel_1850_1900 = (
    f.temperature.sel(layer=0) - 
    f.temperature.sel(layer=0, timebounds=np.arange(1850, 1902)).weighted(weights).mean(dim="timebounds")
).interp(timebounds=f.timepoints)

In [ ]:
temp_rel_1850_1900

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature = (["timepoints", "scenario", "config"], temp_rel_1850_1900.data),
    ),
    coords = dict(
        timepoints = f.timepoints,
        config = f.configs,
        scenario = f.scenarios,
    ),
)

In [ ]:
ds.to_netcdf('../output/temperature-relative-to-1850-1900.nc')

In [ ]:
pl.plot(temp_rel_1850_1900.sel(scenario='ssp245').median(dim='config'), color='r');
pl.plot(temp_rel_1850_1900.sel(scenario='ssp245-ModAggr').median(dim='config'), color='k');
pl.plot(temp_rel_1850_1900.sel(scenario='ssp245-Aggr').median(dim='config'), color='b');
pl.xlim(250, 300)
pl.ylim(0.5, 2.0)

In [ ]:
colors = {
    'ssp245': '#793702',
    'ssp245-ModAggr': '#e3da91',
    'ssp245-Aggr': '#19a5b7'
}

labels = {
    'ssp245': 'Current policies',
    'ssp245-ModAggr': 'Moderate action',
    'ssp245-Aggr': 'Maximum feasibility'
}

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(16/2.54, 5/2.54))

for scenario in scenarios:
    ax[0].plot(
        np.arange(2014.5, 2061), 
        f.emissions.sel(specie='CH4', scenario=scenario, config=f.configs[0], timepoints=np.arange(2014.5, 2061)),
        color=colors[scenario]
    )
ax[0].set_xlim(2015, 2060)
ax[0].set_ylim(0, 420)
ax[0].set_ylabel('Million tons per year')
ax[0].set_title('Methane emissions')

for scenario in scenarios:
    ax[1].fill_between(
        np.arange(2015, 2061), 
        f.concentration.sel(
            specie='CH4',
            scenario=scenario,
            timebounds=np.arange(2015, 2061)
        ).quantile(0.05, dim='config'),
        f.concentration.sel(
            specie='CH4',
            scenario=scenario,
            timebounds=np.arange(2015, 2061)
        ).quantile(0.95, dim='config'),
        color=colors[scenario],
        alpha=0.3
    );
    ax[1].plot(
        np.arange(2015, 2061), 
        f.concentration.sel(specie='CH4', scenario=scenario, timebounds=np.arange(2015, 2061)).median(dim='config'),
        color=colors[scenario],
        label=labels[scenario]
    );
ax[1].set_xlim(2015, 2060)
ax[1].set_ylim(0, 2100)
ax[1].set_ylabel('parts per billion')
ax[1].set_title('Methane concentrations')
ax[1].legend(frameon=False)

for scenario in scenarios:
    ax[2].fill_between(
        np.arange(2014.5, 2061), 
        (
            temp_rel_1850_1900.sel(
                scenario=scenario, 
                timebounds=np.arange(2014.5, 2061)
            ) - temp_rel_1850_1900.sel(
                scenario='ssp245', 
                timebounds=np.arange(2014.5, 2061)
            )
        ).quantile(0.05, dim='config'),
        (
            temp_rel_1850_1900.sel(
                scenario=scenario, 
                timebounds=np.arange(2014.5, 2061)
            ) - temp_rel_1850_1900.sel(
                scenario='ssp245', 
                timebounds=np.arange(2014.5, 2061)
            )
        ).quantile(0.95, dim='config'),
        color=colors[scenario],
        alpha=0.3
    );
    ax[2].plot(
        np.arange(2014.5, 2061), 
        (
            temp_rel_1850_1900.sel(
                scenario=scenario, 
                timebounds=np.arange(2014.5, 2061)
            ) - temp_rel_1850_1900.sel(
                scenario='ssp245', 
                timebounds=np.arange(2014.5, 2061)
            )
        ).median(dim='config'),
        color=colors[scenario]
    );
ax[2].set_xlim(2015, 2060)
ax[2].set_ylim(-0.4, 0.02)
ax[2].set_title('Avoided warming')
ax[2].set_ylabel('°C')

fig.tight_layout()
pl.savefig('../output/emis_conc_temp.png')

In [ ]:
f.forcing.sel(specie='CH4', scenario='ssp245') - f.forcing.sel(specie='CH4', scenario='ssp245-Aggr')
f.forcing.sel(specie='Ozone', scenario='ssp245') - f.forcing.sel(specie='Ozone', scenario='ssp245-Aggr')
f.forcing.sel(specie='Stratospheric water vapour', scenario='ssp245') - f.forcing.sel(specie='Stratospheric water vapour', scenario='ssp245-Aggr')
f.forcing.sel(specie='Aerosol-radiation interactions', scenario='ssp245') - f.forcing.sel(specie='Aerosol-radiation interactions', scenario='ssp245-Aggr')

#f.forcing.sel(specie='Ozone')
#f.forcing.sel(specie='Aerosol-radiation interactions')

In [ ]:
pl.plot(f.forcing_sum.sel(scenario='ssp245').median(dim='config'))
pl.plot(
    (
        f.forcing_sum.sel(scenario='ssp245') - (
            f.forcing.sel(specie='CH4', scenario='ssp245') - 
            f.forcing.sel(specie='CH4', scenario='ssp245-Aggr')
        )
    ).median(dim='config')
)

In [ ]:
forcing_variants = np.ones((382, 8, 841, 1)) * np.nan

inner = ['ssp245-ModAggr', 'ssp245-Aggr']
outer = ['CH4', 'Ozone', 'Stratospheric water vapour', 'Aerosol-radiation interactions']

n_i = 2
n_o = 4

names = []

for i_o, specie in enumerate(outer):
    for i_i, scen in enumerate(inner):
        i = i_o * 2 + i_i
        name = scen + " | " + specie
        names.append(name)
        forcing_variants[:, i, :, 0] = (
            f.forcing_sum.sel(scenario='ssp245') - (
                f.forcing.sel(specie=specie, scenario='ssp245') - 
                f.forcing.sel(specie=specie, scenario=scen)
            )
        ).data

In [ ]:
pl.plot(forcing_variants[:, :, 0, 0])

In [ ]:
names

In [ ]:
g = FAIR()

In [ ]:
g.define_time(1750, 2131, 1)

In [ ]:
g.define_scenarios(names)

In [ ]:
g.define_configs(configs)

In [ ]:
properties

In [ ]:
species = ["forcing"]
properties = {
    "forcing": {
        "type": "unspecified",
        "input_mode": "forcing",
        "greenhouse_gas": False,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
g.define_species(species, properties)

In [ ]:
g.allocate()

In [ ]:
fill(g.forcing, forcing_variants)

In [ ]:
np.sum(np.isnan(g.forcing))

In [ ]:
g.override_defaults(fair_params_1_4_0_file)

In [ ]:
#initialise(f.concentration, f.species_configs["baseline_concentration"])
#initialise(f.forcing, 0)
initialise(g.temperature, 0)
#initialise(f.cumulative_emissions, 0)
#initialise(f.airborne_emissions, 0)
initialise(g.ocean_heat_content_change, 0)

In [ ]:
g.run()

In [ ]:
pl.plot((
    f.temperature.sel(layer=0, scenario='ssp245') - 
    g.temperature.sel(layer=0)
).median(dim="config"))

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature = (
            ["timepoints", "scenario", "config"], 
            (
                f.temperature.sel(layer=0, scenario='ssp245') - 
                g.temperature.sel(layer=0)
            ).interp(timebounds=f.timepoints).data.transpose(0,2,1)
        ),
    ),
    coords = dict(
        timepoints = f.timepoints,
        config = f.configs,
        scenario = names,
    ),
)

In [ ]:
ds.to_netcdf('../output/temperature-contributions.nc')

In [ ]:
(
    f.temperature.sel(layer=0, scenario='ssp245') - 
    g.temperature.sel(layer=0, scenario=names[0])
)